In [1]:
import pandas as pd
import numpy as np
import sklearn
import transformers
import torch
import torch.nn as nn
import torch.functional as f

from transformers import DistilBertTokenizer, DistilBertModel

from sklearn.metrics.pairwise import cosine_similarity

model = DistilBertModel.from_pretrained('bert-base-multilingual-cased')

tokenizer = DistilBertTokenizer.from_pretrained('bert-base-multilingual-cased')

In [5]:
df = pd.read_csv("./df_output4.csv")

In [6]:
df

,video_id,title,channelTitle,view_count,likes,comment_count,description,tags_list,category_title,remain_days,views,like,durations,times,trending_day,uploaded,translations,tags_length
0,UmJiVIRmGxI,손석구에게 납치 시 행동요령에 대해 묻다,피식대학Psick Univ,2003876.0,47573.0,2919.0,피식쇼 조지아크래프트 손석구 피식대학Follow the host of PSICK S...,[],Comedy,13.0,"[2003876.0, 2325702.0, 2524614.0, 2652506.0, 2...","47573.0, 52883.0, 56009.0, 57798.0, 59058.0, 5...",PT34M39S,2079.0,1.0,2023-08-30 09:00:07,"신사 숙녀 여러분 LK 99의 발명 후 가장 위대한 쇼입니다..오, 당신은 멋져 보...",2
1,881P3Vr5GQU,스우파2 2회 바다가 아예 안 보여 리더 계급의 피 튀기는 메인 댄서 선발전 스트릿...,The CHOOM (더 춤),1025631.0,18923.0,2088.0,티빙에서 스트리밍 스우파2 2회 바다가 아예 안 보여 리더 계급의 피 튀기는 메인 ...,"['Mnet', 'SWF2', 'StreetWomanFighter2', 'dance...",Entertainment,5.0,"[1025631.0, 1403948.0, 1720738.0, 1941611.0, 2...","18923.0, 23397.0, 26713.0, 28964.0, 30801.0",PT19M52S,1192.0,1.0,2023-08-31 09:50:33,할로님 어떻게 짰을지 너무이 노래 완전 안무하는 기대된다 안무를 어떻게 표현했을지 ...,93
2,Ec0tKX99gbA,좋아하는 연예인이 고맙다고 나를 안아준다면 뷔 수고했어 오늘도 2023,딩고 스토리 / dingo story,2893689.0,281938.0,22281.0,촬영 날 뷔가 제작진에게 한 말 아미 만나서 놀아보는 게 버킷리스트였어요 슈스 V ...,"['딩고', 'dingo', 'dingostory', '비티에스', '방탄소년단',...",Entertainment,14.0,"[2893689.0, 3420266.0, 4048146.0, 4441726.0, 5...","281938.0, 311781.0, 360313.0, 384382.0, 419764...",PT14M,840.0,1.0,2023-08-31 09:00:08,고마워 뷔가 고맙다고 나를 안아준다면 수고했어 오늘도 2023 수고했어 오늘도 20...,181
3,wYkGQYsXPjI,미국에서 상봉하자마자 오열하는 쿨 유리와 이지혜 여행vlog,밉지않은 관종언니,299472.0,7002.0,438.0,관심이들 까꿍 오늘은 미국 브이로그 마지막편입니다 ㅋㅋ마지막편 답게 정말 탈탈 털어...,"['이지혜', '밉지않은 관종언니', '밉관', '큰태리', '문재완', '이지혜 ...",People & Blogs,15.0,"[299472.0, 380436.0, 452024.0, 509066.0, 56262...","7002.0, 8064.0, 8996.0, 9574.0, 10081.0, 10221...",PT25M36S,1536.0,1.0,2023-08-31 09:15:07,까꿍 샌프란시스코 아마 6일째인가요 나도 이제 헷갈리네 여기는 오늘 코스트코를 왔어...,604
4,HhH2BMHy5PM,탕후루 파트타이머,너덜트,1311714.0,37646.0,2891.0,코믹숏무비 너덜트 극한알바3시간씩 다른 알바를 해야하는 파트타이머 알바생들의 소서사...,"['너덜트', '유현규', '전상협', '임재형', '코믹숏무비', '스케치코미디'...",Comedy,16.0,"[1311714.0, 1786937.0, 2263856.0, 2514904.0, 2...","37646.0, 46399.0, 53507.0, 57034.0, 59048.0, 6...",PT3M57S,237.0,1.0,2023-08-31 12:47:48,지금 몇 개나 갈래 블랙 상파이어야 벌써 준비한게 다 떨어졌어 곧 기사들이 도착할거...,365
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
452,kMuf-L3VgDQ,알타리김치 가장 맛있게 담그는 비법 공개합니다,김대석 셰프TV,105949.0,3934.0,186.0,김대석 셰프의 32년 노하우를 공개합니다 오늘은 초롱무김치 레시피를 알려 드리겠습니...,"['김대석', '김대석셰프', '김대석쉐프', '김대석셰프TV', '김대석쉐프TV'...",Howto & Style,10.0,"[105949.0, 200094.0, 241616.0, 273599.0, 30254...","3934.0, 5677.0, 6456.0, 7078.0, 7567.0, 7914.0...",PT12M57S,777.0,21.0,2023-09-20 23:00:19,안녕하세요 김대석 셰프입니다 오늘은 추석에 먹으면 끝내주는 초록무김치 담아보려고 합...,535
453,I7K5TYPdI08,Q&A,Rena Hattori,196647.0,11852.0,1653.0,_ Rena OFFICIAL SNS 스우파2 TSUBAKILL Rena 레나 스트릿...,[],People & Blogs,6.0,"[196647.0, 272024.0, 501773.0, 513846.0, 52100...","11852.0, 14552.0, 22692.0, 23005.0, 23201.0, 2...",PT16M36S,996.0,21.0,2023-09-20 11:00:11,안녕하세요 여러분 오늘은 무엇을 하시나요 오늘 질문 코너 오늘은 의 이야기에서 다른...,2
454,PBf6a6pth-c,역대급 쓰레기집 청소했습니다,배말랭,293275.0,5374.0,452.0,클린 말랭져스 출동이야 인스타그램 비즈니스 문의 배말랭 배씨표류기 베란다원룸 자취 생존,"['배말랭', '별다먹', '백수', '자취', '핫소스', '생존', '배씨표류기...",People & Blogs,10.0,"[293275.0, 332770.0, 359646.0, 383037.0, 40341...","5374.0, 5783.0, 6073.0, 6289.0, 6466.0, 6550.0...",PT7M56S,476.0,21.0,2023-09-20 11:00:17,뭐야 누가 죽었어 시체성 없는 냄새가 존나 나 집에서 근데 심각하다 진짜 이거 뭐 ...,133
455,PtSS0lQif5w,어느 날 나에게 갑자기 찾아온 루게릭병,삐루빼로,386985.0,21221.0,1248.0,인스타그램 bbiru_unni 인스타그램 bbiru_knitting 문의 루게릭환자...,[],People & Blogs,10.0,"[386985.0, 446943.0, 510297.0, 635268.0, 72900...","21221.0, 22876.0, 24792.0, 26991.0, 28556.0, 2...",PT8M34S,514.0,21.0,2023-09-20 09:54:01,여러분 안녕하세요 오늘은 저와 저희 엄마가 함께 보내는 일상을 조금 자세히 남겨 봤...,2


In [7]:
model.to('cuda')

DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(119547, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0-11): 12 x TransformerBlock(
        (attention): MultiHeadSelfAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2):

In [8]:
dict_titles = {}
for i in range(len(df)):
    try:
        text = df.loc[i, 'title']
        input_ids = tokenizer.encode(text, add_special_tokens=True, max_length=512, padding='max_length', truncation=True, return_tensors='pt').to('cuda')
        outputs = model(input_ids)
        
        embedding = outputs.last_hidden_state.to('cpu')
        
        embed_mtx = embedding.reshape(embedding.shape[0], -1).detach().numpy()
        
        dict_titles[i] = embed_mtx
    except:
        continue
        
    print(i)

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [9]:
dict_titles

{0: array([[ 0.4804911 , -0.50365907, -0.888105  , ...,  1.2757391 ,
          0.92941695,  1.1429203 ]], dtype=float32),
 1: array([[ 0.46635163, -0.51281327, -0.8963378 , ...,  1.2690849 ,
          0.9212532 ,  1.1485469 ]], dtype=float32),
 2: array([[ 0.4640263 , -0.52244335, -0.8886974 , ...,  1.288159  ,
          0.91540366,  1.1605926 ]], dtype=float32),
 3: array([[ 0.46020234, -0.50225204, -0.8835462 , ...,  1.2753664 ,
          0.9386197 ,  1.1414727 ]], dtype=float32),
 4: array([[ 0.46458688, -0.52135783, -0.8841313 , ...,  1.2756469 ,
          0.9292406 ,  1.1505414 ]], dtype=float32),
 5: array([[ 0.47921416, -0.5153307 , -0.88979846, ...,  1.2733316 ,
          0.91859776,  1.1425995 ]], dtype=float32),
 6: array([[ 0.47408473, -0.5145428 , -0.8871634 , ...,  1.2950147 ,
          0.93437886,  1.1563246 ]], dtype=float32),
 7: array([[ 0.4764183, -0.5176504, -0.8730705, ...,  1.2781336,  0.925683 ,
          1.1530887]], dtype=float32),
 8: array([[ 0.45597124, -0.52

In [12]:
cosine_matrix_title = pd.DataFrame()

In [21]:
for i in range(len(dict_titles)):
    for j in range(i, len(dict_titles)):
        cosine_matrix_title.loc[i, j] = float(cosine_similarity(dict_titles[i], dict_titles[j])[0])

C:\Users\dohyeong\AppData\Local\Temp\ipykernel_15956\3681889068.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  cosine_matrix_title.loc[i, j] = float(cosine_similarity(dict_titles[i], dict_titles[j])[0])
C:\Users\dohyeong\AppData\Local\Temp\ipykernel_15956\3681889068.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  cosine_matrix_title.loc[i, j] = float(cosine_similarity(dict_titles[i], dict_titles[j])[0])
C:\Users\dohyeong\AppData\Local\Temp\ipykernel_15956\3681889068.py:3: PerformanceWarning: DataFrame is highly fragment

C:\Users\dohyeong\AppData\Local\Temp\ipykernel_15956\3681889068.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  cosine_matrix_title.loc[i, j] = float(cosine_similarity(dict_titles[i], dict_titles[j])[0])
C:\Users\dohyeong\AppData\Local\Temp\ipykernel_15956\3681889068.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  cosine_matrix_title.loc[i, j] = float(cosine_similarity(dict_titles[i], dict_titles[j])[0])
C:\Users\dohyeong\AppData\Local\Temp\ipykernel_15956\3681889068.py:3: PerformanceWarning: DataFrame is highly fragment

C:\Users\dohyeong\AppData\Local\Temp\ipykernel_15956\3681889068.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  cosine_matrix_title.loc[i, j] = float(cosine_similarity(dict_titles[i], dict_titles[j])[0])
C:\Users\dohyeong\AppData\Local\Temp\ipykernel_15956\3681889068.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  cosine_matrix_title.loc[i, j] = float(cosine_similarity(dict_titles[i], dict_titles[j])[0])
C:\Users\dohyeong\AppData\Local\Temp\ipykernel_15956\3681889068.py:3: PerformanceWarning: DataFrame is highly fragment

C:\Users\dohyeong\AppData\Local\Temp\ipykernel_15956\3681889068.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  cosine_matrix_title.loc[i, j] = float(cosine_similarity(dict_titles[i], dict_titles[j])[0])
C:\Users\dohyeong\AppData\Local\Temp\ipykernel_15956\3681889068.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  cosine_matrix_title.loc[i, j] = float(cosine_similarity(dict_titles[i], dict_titles[j])[0])
C:\Users\dohyeong\AppData\Local\Temp\ipykernel_15956\3681889068.py:3: PerformanceWarning: DataFrame is highly fragment

C:\Users\dohyeong\AppData\Local\Temp\ipykernel_15956\3681889068.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  cosine_matrix_title.loc[i, j] = float(cosine_similarity(dict_titles[i], dict_titles[j])[0])
C:\Users\dohyeong\AppData\Local\Temp\ipykernel_15956\3681889068.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  cosine_matrix_title.loc[i, j] = float(cosine_similarity(dict_titles[i], dict_titles[j])[0])
C:\Users\dohyeong\AppData\Local\Temp\ipykernel_15956\3681889068.py:3: PerformanceWarning: DataFrame is highly fragment

In [19]:
cosine_matrix_title.loc[1, 2] = float(cosine_similarity(dict_titles[1], dict_titles[2])[0])

In [ ]:
text = df.loc[1, 'title']
input_ids = tokenizer.encode(text, add_special_tokens=True, max_length=512, padding='max_length', truncation=True, return_tensors='pt')
outputs = model(input_ids)

embedding = outputs.last_hidden_state

embed_mtx = embedding.reshape(embedding.shape[0], -1).detach().numpy()

dict_titles[1] = embed_mtx

In [25]:
cosine_matrix_title.loc[1, :]

2      0.972156
0           NaN
1      0.999999
3      0.973497
4      0.978384
         ...   
452    0.974779
453    0.979592
454    0.976682
455    0.975936
456    0.963162
Name: 1, Length: 457, dtype: float64

In [29]:
mat0 = cosine_matrix_title.loc[:, 0]

In [30]:
mat2 = cosine_matrix_title.loc[:, 2]

In [34]:
cos_mat2 = cosine_matrix_title

In [35]:
for i in range(len(df)):
    for j in range(len(df)):
        if cosine_matrix_title.loc[i, j] >= 0:
            continue
        else:
            cosine_matrix_title.loc[i, j] = cosine_matrix_title.loc[j, i]

In [43]:
cosine_matrix_title = cosine_matrix_title[cols]

In [65]:
cosine_matrix_title.to_csv("cos_sim_title.csv")